In [1]:
import sys
import os
sys.path.append("../src/whole_history_rating")
from whr import whole_history_rating
from whr import utils
from datetime import datetime
import datetime as dt
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
data_path = '../data/cfbd/games_1990_to_2020.csv'

In [3]:
games_df = pd.read_csv(data_path)
games_df_test = games_df[games_df.season>2015]
games_df_verify = games_df[games_df.season>2010]
games_df.shape

(23256, 25)

In [4]:
datetime_from_cfbd = lambda x: datetime.strptime(x,"%Y-%m-%dT%H:%M:%S.%fZ")

In [5]:

def benchmark_w2(default_w2, verbose=False, time_offset=700):
    correct_prediction_list = []
    delta_mov_list = []
    scale = 13
    w2 = (300*default_w2**2)/(400**2)
    whr = whole_history_rating.Base({'scale': scale,'w2':w2, 'debug':False})
    start_time = 0
    home_team_handicap = 2.5/13*scale
    current_day,day_delta=0,0
    for i,row in games_df_verify.iterrows():
        game_time = datetime_from_cfbd(row.start_date)
        if start_time == 0: start_time = game_time
        day_delta = (game_time-start_time).days
        
        if current_day!=day_delta: 
            print("new day: {}".format(day_delta))
            if len(correct_prediction_list)>0: print("correct: {:.1f}%".format(np.mean(correct_prediction_list)*100 ))
            if len(delta_mov_list)>0: print("abe: {:.2f}".format( np.mean(np.abs(delta_mov_list)) ))
            whr.auto_iterate(time_limit = 10, precision = 10E-4*scale/400)
            current_day=day_delta

        home_team = row.home_team
        away_team = row.away_team
        handicap = -(1-row.neutral_site)*home_team_handicap
        mov = row.home_points-row.away_points
        winner = "W" if mov>0 else "B"
        if mov==0: continue
        home_w_prob,away_w_prob = whr.probability_future_match(home_team,away_team,-handicap, verbose=False)
        home_rating,home_unc = whr.ratings_for_player(home_team,current=True)
        away_rating,away_unc = whr.ratings_for_player(away_team,current=True)
        
        if home_rating and away_rating:
            predicted_mov = home_rating-away_rating+handicap
        else:
            predicted_mov = None
            
        correct_prediction = (home_w_prob-away_w_prob)*mov>0
        if time_offset < day_delta: 
            correct_prediction_list.append(correct_prediction)
            if predicted_mov: delta_mov_list.append(predicted_mov)
            
        if verbose: 
            print("\t away team: {}, home team: {}".format(away_team,home_team))
            print("\t winner: {} (W is home), days: {} handicap: {}".format(winner,day_delta,handicap))
            print("\t {} win: {:.2f}%, predicted spread: {}, actual spread: {}".format(home_team,home_w_prob*100,predicted_mov, mov))
            print("\t day_delta: {} correct prediction: {}".format(day_delta,correct_prediction))
            print(home_w_prob,away_w_prob)
            print(home_rating,away_rating)
        if time_offset < day_delta:
            whr.create_game(away_team,home_team,winner, day_delta, handicap)
        
        #if i > 100: break
    return correct_prediction_list

In [6]:
correct_prediction_list = benchmark_w2(300)
np.average(correct_prediction_list)

new day: 1
new day: 2
new day: 3
new day: 4
new day: 7
new day: 8
new day: 9
new day: 14
new day: 15
new day: 16
new day: 21
new day: 22
new day: 23
new day: 28
new day: 29
new day: 30
new day: 35
new day: 36
new day: 37
new day: 42
new day: 43
new day: 44
new day: 47
new day: 49
new day: 50
new day: 51
new day: 54
new day: 55
new day: 56
new day: 57
new day: 58
new day: 61
new day: 62
new day: 63
new day: 64
new day: 65
new day: 68
new day: 69
new day: 70
new day: 71
new day: 72
new day: 75
new day: 76
new day: 77
new day: 78
new day: 79
new day: 82
new day: 84
new day: 85
new day: 86
new day: 91
new day: 92
new day: 93
new day: 99
new day: 106
new day: 107
new day: 110
new day: 111
new day: 112
new day: 114
new day: 115
new day: 116
new day: 117
new day: 118
new day: 119
new day: 120
new day: 121
new day: 122
new day: 123
new day: 124
new day: 125
new day: 127
new day: 129
new day: 130
new day: 364
new day: 365
new day: 366
new day: 367
new day: 368
new day: 371
new day: 372
new day:

../src/whole_history_rating/whr/playerday.py:67: RuntimeWarning: divide by zero encountered in double_scalars
  result += (c * d) / ((c * self.gamma() + d) ** 2.0)
../src/whole_history_rating/whr/playerday.py:68: RuntimeWarning: invalid value encountered in double_scalars
  return -1 * self.gamma() * result


AttributeError: bad adjusted gamma

In [ ]:
correct_prediction_list